In [28]:
# import packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt
from sklearn import linear_model
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
import geopandas as gpd


import lightgbm as lgb
from sklearn.metrics import mean_squared_error

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import dask.dataframe as dd
#from dask_ml.datasets import make_regression

from sklearn.model_selection import train_test_split



import math
from scipy import stats
from sklearn.linear_model import LinearRegression


from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs

from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

%matplotlib inline

#import ArcgisLib

# suppress warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [29]:
import swifter

In [30]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [32]:
X_train.shape

(5389800, 17)

In [33]:
X_train = np.delete(X_train, 15, 1)
X_train.shape

(5389800, 16)

In [34]:
X_test.shape

(1347450, 17)

In [35]:
X_test = np.delete(X_test, 15, 1)
X_test.shape

(1347450, 16)

In [36]:
training_data = pd.DataFrame(np.concatenate((y_train.reshape(len(y_train),1),X_train),axis=1))
training_data.columns=["y"]+["x{}".format(i) for i in range(1,17)]

test_data = pd.DataFrame(np.concatenate((y_test.reshape(len(y_test),1),X_test),axis=1))
test_data.columns=["y"]+["x{}".format(i) for i in range(1,17)]

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.1,
                        n_estimators=10)

In [39]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)

[1]	valid_0's l1: 510776	valid_0's l2: 9.35587e+12
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 492772	valid_0's l2: 8.64666e+12
[3]	valid_0's l1: 477378	valid_0's l2: 8.0445e+12
[4]	valid_0's l1: 461923	valid_0's l2: 7.51673e+12
[5]	valid_0's l1: 452627	valid_0's l2: 7.09893e+12
[6]	valid_0's l1: 439394	valid_0's l2: 6.58776e+12
[7]	valid_0's l1: 425816	valid_0's l2: 6.20855e+12
[8]	valid_0's l1: 416797	valid_0's l2: 5.93158e+12
[9]	valid_0's l1: 410373	valid_0's l2: 5.56661e+12
[10]	valid_0's l1: 403417	valid_0's l2: 5.35763e+12
Did not meet early stopping. Best iteration is:
[10]	valid_0's l1: 403417	valid_0's l2: 5.35763e+12


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=10, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [40]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 2314655.0765316007


In [41]:
print('Feature importances:', list(gbm.feature_importances_))

Feature importances: [35, 3, 27, 24, 73, 33, 40, 22, 19, 2, 1, 3, 3, 1, 5, 9]


In [42]:
def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

In [43]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=rmsle,
        early_stopping_rounds=5)

[1]	valid_0's l2: 9.35587e+12	valid_0's RMSLE: 0.940798
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 8.64666e+12	valid_0's RMSLE: 0.921691
[3]	valid_0's l2: 8.0445e+12	valid_0's RMSLE: 0.907722
[4]	valid_0's l2: 7.51673e+12	valid_0's RMSLE: 0.887717
[5]	valid_0's l2: 7.09893e+12	valid_0's RMSLE: 0.879933


C:\Users\aniha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


[6]	valid_0's l2: 6.58776e+12	valid_0's RMSLE: nan
[7]	valid_0's l2: 6.20855e+12	valid_0's RMSLE: nan
[8]	valid_0's l2: 5.93158e+12	valid_0's RMSLE: nan
[9]	valid_0's l2: 5.56661e+12	valid_0's RMSLE: nan
[10]	valid_0's l2: 5.35763e+12	valid_0's RMSLE: nan
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.35763e+12	valid_0's RMSLE: nan


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=10, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [44]:
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False

In [45]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=lambda y_true, y_pred: [rmsle(y_true, y_pred), rae(y_true, y_pred)],
        early_stopping_rounds=5)

[1]	valid_0's l2: 9.35587e+12	valid_0's RMSLE: 0.940798	valid_0's RAE: 0.954632
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 8.64666e+12	valid_0's RMSLE: 0.921691	valid_0's RAE: 0.920983
[3]	valid_0's l2: 8.0445e+12	valid_0's RMSLE: 0.907722	valid_0's RAE: 0.892213
[4]	valid_0's l2: 7.51673e+12	valid_0's RMSLE: 0.887717	valid_0's RAE: 0.863327
[5]	valid_0's l2: 7.09893e+12	valid_0's RMSLE: 0.879933	valid_0's RAE: 0.845953


C:\Users\aniha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


[6]	valid_0's l2: 6.58776e+12	valid_0's RMSLE: nan	valid_0's RAE: 0.82122
[7]	valid_0's l2: 6.20855e+12	valid_0's RMSLE: nan	valid_0's RAE: 0.795843
[8]	valid_0's l2: 5.93158e+12	valid_0's RMSLE: nan	valid_0's RAE: 0.778987
[9]	valid_0's l2: 5.56661e+12	valid_0's RMSLE: nan	valid_0's RAE: 0.76698
[10]	valid_0's l2: 5.35763e+12	valid_0's RMSLE: nan	valid_0's RAE: 0.753979
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.35763e+12	valid_0's RMSLE: nan	valid_0's RAE: 0.753979


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=10, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [46]:
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmsle of prediction is:', rmsle(y_test, y_pred)[1])
print('The rae of prediction is:', rae(y_test, y_pred)[1])

C:\Users\aniha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


The rmsle of prediction is: nan
The rae of prediction is: 0.753979545066449


In [47]:
# other scikit-learn modules
estimator = lgb.LGBMRegressor(num_leaves=31)

In [48]:
param_grid = {
    'learning_rate': np.arange(0.001, 1, 0.01),
    'n_estimators': np.arange(40, 300, 10)
}

In [11]:
gbm = GridSearchCV(estimator, param_grid, cv=4, verbose=1)
gbm.fit(X_train, y_train)

Fitting 4 folds for each of 2600 candidates, totalling 10400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print('Best parameters found by grid search are:', gbm.best_params_)